<h1>Segmenting-and-Clustering-Neighborhoods-in-Toronto </h1>
<h2>Part 1</h2>

***
- For this assignment, you will be required to explore and cluster the neighborhoods in Toronto. <br>
    1. Start by creating a new Notebook for this assignment. <br>
    2. Use the Notebook to build the code to scrape the following Wikipedia page,<br> 
        https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe <br>
    3. To create the dataframe: <br>
        * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood <br>
        * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. <br>
        * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods:<br>
            Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table. <br>
        * If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough. <br>
        * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
        * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
    4. Submit a link to your Notebook on your Github repository. 
***

In [119]:
##import everything
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import requests # library to handle requests
from bs4 import BeautifulSoup #library to scrape data

print('Libraries imported.')

Libraries imported.


<h4>Part 1 - Q2/Q3:downlaod, scrape, and wrangling</h4>

In [120]:
#Scrape the page!!!
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL)
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table')

#develop dataframe
df = pd.read_html(str(table))[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [121]:
#rename as desired
df.columns = ['PostalCode','Borough','Neighborhood']
# Replace "Borough" with 'Not assigned' values
df2=df[df['Borough'].str.contains("Not assigned") == False]
df2.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [122]:
#group as desired
df3= df2.groupby(['PostalCode','Borough'], sort=False)['Neighborhood'].apply(', '.join).reset_index()
print("df shape is: ", df3.shape)
df3.head()

df shape is:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h2>Part 2</h2>

***
- Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, <br>
    in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. <br>
- Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository.<br>
***

In [123]:
df_geo_url="https://cocl.us/Geospatial_data"
df_geo=pd.read_csv(df_geo_url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [124]:
df_geo.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_toronto = pd.merge(df3,df_geo,on='PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h2>Part 3</h2>

***
- Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then <br>
    replicate the same analysis we did to the New York City data. It is up to you. 
- Just make sure:
    1. to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
    2. to generate maps to visualize your neighborhoods and how they cluster together. 
***

In [126]:
##Throw in foursquare API credemtials
CODE = 'LXAZ13AJWVQWTJZZV0STVZLRVVVJ33JU4XM0J3YS3OPL3AR5#_=_'
CLIENT_ID = '4N3EJOXQVAHJBWOASRSK12CVR0G4PTWJ0THHQ3H5JYLEYW5B' # your Foursquare ID
CLIENT_SECRET = '05UGKRHWE51ECB553ZXP4VWIJWZUKZUNJK0KVIUQQGYAFRNV' # your Foursquare Secret
ACCESS_TOKEN = 'YJVT4ZQZPSXV4WL3DQSEEWYYQT21VE2YJP5ZWZCHQPKUZTMB' # your FourSquare Access Token
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4N3EJOXQVAHJBWOASRSK12CVR0G4PTWJ0THHQ3H5JYLEYW5B
CLIENT_SECRET:05UGKRHWE51ECB553ZXP4VWIJWZUKZUNJK0KVIUQQGYAFRNV


In [140]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import folium # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print('Libraries imported.')

Libraries imported.


In [141]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.6534817, -79.3839347.


In [142]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [143]:
#cluster each neighborhoood on the map
k=5
toronto_clustering = df_toronto.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df_toronto.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [146]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, Neighborhood, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'],df_toronto['Neighborhood'],df_toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters